In [ ]:
%sql
-- SQL for calculating applied quantities in the inventory transactions context

-- Create a CTE to calculate the applied quantity based on specified conditions
WITH calculated_apl_qty AS (
  SELECT 
    txn_id,
    ref_txn_qty,
    cumulative_txn_qty,
    cumulative_ref_ord_sched_qty,
    ref_ord_sched_qty,
    prior_cumulative_txn_qty,
    prior_cumulative_ref_ord_sched_qty,
    CASE
      -- Condition 1: ref_txn_qty > 0 and cumulative_txn_qty >= cumulative_ref_ord_sched_qty
      WHEN ref_txn_qty > 0 
           AND cumulative_txn_qty >= cumulative_ref_ord_sched_qty 
           AND prior_cumulative_txn_qty > prior_cumulative_ref_ord_sched_qty THEN
        -- Calculate apl_qty for Condition 1
        ref_ord_sched_qty - (prior_cumulative_txn_qty - prior_cumulative_ref_ord_sched_qty)
      
      -- Condition 2: ref_txn_qty > 0 and cumulative_ref_ord_sched_qty >= cumulative_txn_qty
      WHEN ref_txn_qty > 0 
           AND cumulative_ref_ord_sched_qty >= cumulative_txn_qty 
           AND prior_cumulative_ref_ord_sched_qty > prior_cumulative_txn_qty THEN
        -- Calculate apl_qty for Condition 2
        ref_txn_qty - (prior_cumulative_ref_ord_sched_qty - prior_cumulative_txn_qty)
      
      -- Condition 3: ref_txn_qty < 0, cumulative_txn_qty != 0, and cumulative_ref_ord_sched_qty > 0
      WHEN ref_txn_qty < 0 
           AND cumulative_txn_qty != 0 
           AND cumulative_ref_ord_sched_qty > 0 THEN
        -- Direct assignment for Condition 3
        ref_txn_qty
      
      -- Default condition: apply NULL if none of the conditions match
      ELSE NULL
    END AS apl_qty
  FROM purgo_playground.f_inv_movmnt_apl_qty_test  -- Using the test dataset
)

-- Validate and output the calculated data for confirmation and debug purposes
SELECT * FROM calculated_apl_qty;

-- Performance optimization: evaluate execution plan
EXPLAIN
SELECT apl_qty, COUNT(*) 
FROM calculated_apl_qty 
GROUP BY apl_qty;

-- Data Quality Check: count rows with NULL apl_qty where conditions should apply
SELECT COUNT(*) AS null_count
FROM calculated_apl_qty
WHERE apl_qty IS NULL 
  AND (ref_txn_qty > 0 OR ref_txn_qty < 0);

